# Biosemi Active II recorded file streaming

In [7]:
# import mne
# import numpy as np
# from mne.channels import make_standard_montage
# import matplotlib.pyplot as plt
# from mne.datasets import eegbci

# raw = mne.io.read_raw_bdf("C:\\Users\\pipo_\\OneDrive\\Desktop\\test_data\\Testdata3.bdf")

# eegbci.standardize(raw)  # set channel names
# montage = make_standard_montage("biosemi64")

# raw.set_montage(montage, on_missing='ignore')

# raw_eeg = raw.copy().resample(512)

# raw_eeg.save("biosemi_SSVEP.fif", overwrite=True)

In [8]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci


raw_eeg = mne.io.read_raw_fif("biosemi_SSVEP.fif")

Opening raw data file biosemi_SSVEP.fif...
    Range : 0 ... 100863 =      0.000 ...   196.998 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_7988\4293364053.py:8: RuntimeWarning: This filename (biosemi_SSVEP.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif("biosemi_SSVEP.fif")


In [9]:
raw_eeg

<Raw | biosemi_SSVEP.fif, 73 x 100864 (197.0 s), ~87 kB, data not loaded>

In [10]:
events_raw = mne.find_events(raw_eeg, shortest_event = 0)

event_dict =  {'12Hz': 8,
'24Hz': 4,
'6Hz': 10,
'30Hz': 2,
'trial_begin': 1,
'trial_end': 20,
'end_exper': 40
}

Trigger channel STATUS has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
61 events found on stim channel STATUS
Event IDs: [ 1  2  4  8 10 20 40]


In [11]:
raw_eeg.ch_names

['Fp1',
 'AF7',
 'AF3',
 'F1',
 'F3',
 'F5',
 'F7',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'C1',
 'C3',
 'C5',
 'T7',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'P1',
 'P3',
 'P5',
 'P7',
 'P9',
 'PO7',
 'PO3',
 'O1',
 'Iz',
 'Oz',
 'POz',
 'Pz',
 'CPz',
 'Fpz',
 'Fp2',
 'AF8',
 'AF4',
 'AFz',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT8',
 'FC6',
 'FC4',
 'FC2',
 'FCz',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP8',
 'CP6',
 'CP4',
 'CP2',
 'P2',
 'P4',
 'P6',
 'P8',
 'P10',
 'PO8',
 'PO4',
 'O2',
 'EXG1',
 'EXG2',
 'EXG3',
 'EXG4',
 'EXG5',
 'EXG6',
 'EXG7',
 'EXG8',
 'STATUS']

In [12]:
from mne_realtime import LSLClient, MockLSLStream
from pylsl import StreamInfo, StreamOutlet
import time
import mne

# Assuming raw_eeg is already created
# Create the LSL stream for the raw EEG data
eeg_info = StreamInfo(name='SomSom2', type='EEG', channel_count=raw_eeg.info['nchan'],
                      nominal_srate=raw_eeg.info['sfreq'], channel_format='float32')
eeg_outlet = StreamOutlet(eeg_info)

def send_eeg_chunks(raw_eeg, chunk_size):
    """ Send raw EEG data in chunks through LSL. """
    sfreq = raw_eeg.info['sfreq']  # Sampling frequency
    n_samples = len(raw_eeg.times)  # Total number of samples in the raw data
    data, _ = raw_eeg[:, :]  # Get all the EEG data as an array

    for start_idx in range(0, n_samples, chunk_size):
        # Determine the end of the chunk
        end_idx = min(start_idx + chunk_size, n_samples)
        
        # Extract the chunk of data
        chunk = data[:, start_idx:end_idx]

        # print(chunk.shape)

        # Push the chunk of raw EEG data to the LSL stream
        eeg_outlet.push_chunk(chunk.T.tolist())  # Transpose for proper LSL format
        # print(f"Sent chunk of {end_idx - start_idx} samples")

        time.sleep(1/sfreq)

if __name__ == '__main__':
    while True:
        # Set the chunk size (e.g., 100 samples per chunk)
        chunk_size = 512
        # Send raw EEG data in chunks
        send_eeg_chunks(raw_eeg, chunk_size)

KeyboardInterrupt: 

In [37]:
# from mne_realtime import LSLClient, MockLSLStream
# from pylsl import StreamInfo, StreamOutlet
# import time
# import mne

# # Assuming raw_eeg is already created and events are detected
# # Find events in raw_eeg (already done in your case)
# events_raw = mne.find_events(raw_eeg, shortest_event=0)

# # Create the LSL stream for the trigger events (Markers)
# trigger_info = StreamInfo(name='TriggerStream', type='Markers', channel_count=1, nominal_srate=raw_eeg.info['sfreq'], channel_format='int32')
# trigger_outlet = StreamOutlet(trigger_info)

# # Host name and wait time
# host = 'My_mne_stream'
# wait_max = 5

# previous_event_time = 0

# def send_triggers(events_raw, sfreq):
#     global previous_event_time
#     """ Send trigger events through LSL based on event timestamps. """
#     for event in events_raw:
#         sample_idx = event[0]  # The sample index of the event
#         trigger_type = event[2]  # The event ID (trigger type)

#         # Convert the sample index to time in seconds
#         event_time = (sample_idx / sfreq)
        
#         # Simulate waiting until the event time
#         print("waiting to send event for: ",event_time-previous_event_time)
#         time.sleep(event_time-previous_event_time)
        
#         # Send the trigger over LSL
#         trigger_outlet.push_sample([trigger_type])
#         print(f"Sent trigger: {trigger_type} at time {event_time+previous_event_time:.3f} seconds")

#         previous_event_time = event_time

# if __name__ == '__main__':
#     # Start the EEG stream using MockLSLStream
#     with MockLSLStream(host, raw_eeg, 'eeg'):
#         # Start the client to receive the EEG stream
#         with LSLClient(info=raw_eeg.info, host=host, wait_max=wait_max) as client:
#             client_info = client.get_measurement_info()
            
#             # Send triggers in real-time based on the events found in the raw data
#             send_triggers(events_raw, raw_eeg.info['sfreq'])

#             # Infinite loop to simulate real-time streaming
#             while True:
#                 pass  # Keeps the stream alive


Trigger channel STATUS has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.


61 events found on stim channel STATUS
Event IDs: [ 1  2  4  8 10 20 40]
Reading 0 ... 100863  =      0.000 ...   196.998 secs...
Client: Waiting for server to start
Looking for LSL stream My_mne_stream...
Found stream 'MNE' via My_mne_stream...
Client: Connected
waiting to send event for:  18.013671875
Sent trigger: 1 at time 18.014 seconds
waiting to send event for:  1.0
Sent trigger: 8 at time 37.027 seconds
waiting to send event for:  5.0
Sent trigger: 20 at time 43.027 seconds
waiting to send event for:  2.001953125
Sent trigger: 1 at time 50.029 seconds
waiting to send event for:  1.001953125
Sent trigger: 2 at time 53.033 seconds
waiting to send event for:  5.0
Sent trigger: 20 at time 59.035 seconds
waiting to send event for:  2.0
Sent trigger: 1 at time 66.035 seconds
waiting to send event for:  1.001953125
Sent trigger: 4 at time 69.037 seconds
waiting to send event for:  5.001953125
Sent trigger: 20 at time 75.041 seconds
waiting to send event for:  2.0
Sent trigger: 1 at ti

KeyboardInterrupt: 